# Import needed libraries

In [12]:
from statistics import mean,stdev
from termcolor import colored
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
import collections
import heapq
import operator
import networkx as nx
import os
import os.path
import random
import numpy as np
from os import walk
from os import path
import sys
from utils import *
from variables import *

# Functions

In [13]:
def get_top_k_best_nodes(dict_centrality, k):
    first_nodes = heapq.nlargest(k, dict_centrality, key=dict_centrality.get)    
    return first_nodes

In [14]:
def get_bottom_k_best_nodes(dict_centrality, k):
    bottom_nodes = heapq.nsmallest(k, dict_centrality, key=dict_centrality.get)   
    return bottom_nodes

In [15]:
def nth_moment(g,n):
    degree_np = np.array(list(dict(g.degree).values()))
    return (sum(degree_np**n)/len(g))

In [16]:
cleanNetworks = '/SyntheticNetworks'
networks = ['networkovermu035.gml',
 'networkovermu030.gml',
 'networkovermu015.gml',
 'networkovermu010.gml',
 'networkovermu040.gml',
 'networkovermu005.gml',
 'networkovermu070.gml',
 'networkovermu025.gml',
 'networkovermu020.gml']

# IC Simulations for All Networks

In [18]:
#networks = next(walk(cleanNetworks), (None, None, []))[2]

for network in networks:
    counter = 0
    g = nx.read_gml(f'{cleanNetworks}/{network}',destringizer=int)
    n = network.split(".")[0]
    
    print(colored(f'Simulation currently on network {network} : {nx.info(g)}...', 'red',attrs=['bold']))
    #Remove Self Loops
    g.remove_edges_from(nx.selfloop_edges(g))
    
    #Take Largest Component
    nb_components = nx.number_connected_components(g)
    if(nb_components>1):
        subgraphs = list(nx.connected_components(g))
        max = subgraphs[0]
        for subgraph in subgraphs:
            if len(subgraph)>len(max):
                max = subgraph
        g = g.subgraphs(max)
    #print(network)
    
    #Initialize a dictionary for each centrality
    list_dicts = []

    for centrality in colors.keys():
        globals()[f'dict_{centrality}'] = dict()
        list_dicts.append(globals()[f'dict_{centrality}'])
        
    #get centrality results
    for dictionary,centrality in zip(list_dicts,colors.keys()):
        p = f'/SyntheticNetworks/centrality_results/{n}/dict_{centrality}.txt'
        with open(p) as raw_data:
            for item in raw_data:
                if ':' in item:
                    key,value = item.split(':', 1)
                    value = value.replace('\n', '')
                    dictionary[int(key)]=float(value)
                else:
                    pass # deal with bad lines of text here
        #print(dictionary)
    
    #Initialize Average and Standard Deviation for Each Centrality
    for centrality in colors.keys():
        globals()[f'{centrality}_average_each_simulation'] = dict()
        globals()[f'{centrality}_standard_each_simulation'] = dict()
        for fraction in fractions: 
            globals()[f'{centrality}_average_each_simulation'][fraction] = 0
            globals()[f'{centrality}_standard_each_simulation'][fraction] = 0
    
    p = f'./Pickle Variables/{n}/'
    if(path.exists(p)==False):
        os.mkdir(p)
    network_size = len(g.nodes())
    save_obj(network_size,p,'networkSize')
    
    #Epidemic threshold
    firstMoment = nth_moment(g,1)
    secondMoment = nth_moment(g,2)
    epidemicThreshold = round(firstMoment/(secondMoment-firstMoment),3)
    save_obj(epidemicThreshold,p,'epidemicThreshold')
    
    #thresholds
    randomThresholds = np.random.uniform(low=0, high=1, size=len(g.nodes()))
    fixedThresholds = [epidemicThreshold]
    
    #Run diffusion model on centralities
    for counter in range(2): 
        for centrality in colors.keys():
            print(colored(f'Counter: {counter}, Centrality: {centrality}', 'green',attrs=['bold']))
            for fraction in fractions:
                globals()[f'l_{centrality}'] = list()
                fraction_infected = round(fraction*network_size)

                if(centrality=="modv_neg"):
                    infected_nodes = get_bottom_k_best_nodes(globals()[f'dict_{centrality}'], fraction_infected)
                else:
                    infected_nodes = get_top_k_best_nodes(globals()[f'dict_{centrality}'], fraction_infected)

                for i in range(repeatedSimulations): 

                    model = ep.IndependentCascadesModel(g)
                    config = mc.Configuration()

                    if(counter==0):
                        thresholdType = f'Threshold-{fixedThresholds[counter]}'
                        for e in g.edges():
                            config.add_edge_configuration("threshold", e, fixedThresholds[counter])  
                    else:
                        thresholdType = 'UniformRandomThresholds'
                        random.shuffle(randomThresholds)
                        for e,threshold in zip(g.edges(),randomThresholds):
                            config.add_edge_configuration("threshold", e, threshold)

                    #config.add_model_parameter('fraction_infected', fraction_infected)
                    config.add_model_initial_configuration('Infected', infected_nodes)
                    model.set_initial_status(config)

                    x = 0
                    iterations = model.iteration_bunch(1)
           
                    while((iterations[x]['node_count'][1])!=0):
                        x = x + 1
                        iterations = iterations + model.iteration_bunch(1)

                    finalRecoveredInOneIteration = iterations[x]['node_count'][2]
                    #print(finalRecoveredInOneIteration)
                    globals()[f'l_{centrality}'].append(finalRecoveredInOneIteration)

                print(fraction,":",mean(globals()[f'l_{centrality}']))
                globals()[f'{centrality}_average_each_simulation'][fraction] = mean(globals()[f'l_{centrality}'])
                globals()[f'{centrality}_standard_each_simulation'][fraction] = stdev(globals()[f'l_{centrality}'])

            p = f"./Pickle Variables/{n}/{thresholdType}/"
            if(path.exists(p)==False):
                os.mkdir(p)
            save_obj(globals()[f'{centrality}_average_each_simulation'], p ,f'{centrality}_average_each_simulation')
            save_obj(globals()[f'{centrality}_standard_each_simulation'], p , f'{centrality}_standard_each_simulation')    

Simulation currently on network networkovermu035.gml : Name: 
Type: Graph
Number of nodes: 2500
Number of edges: 10417
Average degree:   8.3336...
Counter: 0, Centrality: modv_pos
0.01 : 233.9
0.02 : 348.57
0.03 : 417.26
0.04 : 524.1
0.05 : 568.21
0.06 : 621.8
0.07 : 670.53
0.08 : 708.81
0.09 : 747.28
0.1 : 790.44
0.11 : 839.01
0.12 : 867.41
0.13 : 915.55
0.14 : 957.96
0.15 : 992.05
0.16 : 1026.99
0.17 : 1060.2
0.18 : 1101.5
0.19 : 1125.1
0.2 : 1168.38
0.21 : 1196.42
0.22 : 1221.91
0.23 : 1249.37
0.24 : 1278.29
0.25 : 1297.69
0.26 : 1329.19
0.27 : 1358.8
0.28 : 1376.77
0.29 : 1399.77
0.3 : 1425.24
0.31 : 1452.32
0.32 : 1476.11
0.33 : 1498.03
0.34 : 1521.16
0.35 : 1542.53
0.36 : 1564.64
0.37 : 1585.89
0.38 : 1603.96
0.39 : 1628.26
0.4 : 1646.29
0.41 : 1669.82
0.42 : 1682.06
0.43 : 1704.98
0.44 : 1722.71
0.45 : 1738.51
0.46 : 1753.47
0.47 : 1775.29
0.48 : 1788.7
0.49 : 1801.03
0.5 : 1814.02
Counter: 0, Centrality: com
0.01 : 176.68
0.02 : 298.18
0.03 : 397.13
0.04 : 475.41
0.05 : 565.79


0.26 : 1161.79
0.27 : 1182.7
0.28 : 1193.89
0.29 : 1215.25
0.3 : 1238.21
0.31 : 1265.23
0.32 : 1287.41
0.33 : 1303.15
0.34 : 1320.88
0.35 : 1333.58
0.36 : 1353.04
0.37 : 1375.79
0.38 : 1396.54
0.39 : 1417.33
0.4 : 1447.85
0.41 : 1473.11
0.42 : 1483.31
0.43 : 1499.81
0.44 : 1513.69
0.45 : 1528.08
0.46 : 1550.85
0.47 : 1569.49
0.48 : 1583.86
0.49 : 1605.59
0.5 : 1627.78
Counter: 1, Centrality: modv_pos
0.01 : 139.95
0.02 : 244.54
0.03 : 343.5
0.04 : 429.46
0.05 : 504.18
0.06 : 577.41
0.07 : 646.18
0.08 : 697.37
0.09 : 756.77
0.1 : 811.7
0.11 : 855.89
0.12 : 919.43
0.13 : 969.83
0.14 : 1014.25
0.15 : 1050.26
0.16 : 1083.77
0.17 : 1123.79
0.18 : 1163.82
0.19 : 1184.35
0.2 : 1221.42
0.21 : 1241.09
0.22 : 1273.8
0.23 : 1305.74
0.24 : 1329.24
0.25 : 1369.13
0.26 : 1390.05
0.27 : 1404.79
0.28 : 1424.72
0.29 : 1447.61
0.3 : 1469.66
0.31 : 1499.98
0.32 : 1522.59
0.33 : 1539.3
0.34 : 1564.78
0.35 : 1583.86
0.36 : 1605.61
0.37 : 1626.98
0.38 : 1650.24
0.39 : 1672.43
0.4 : 1691.91
0.41 : 1712.58
0.

0.12 : 663.7
0.13 : 700.92
0.14 : 736.66
0.15 : 764.82
0.16 : 809.99
0.17 : 840.94
0.18 : 871.23
0.19 : 905.13
0.2 : 948.79
0.21 : 975.35
0.22 : 997.97
0.23 : 1025.17
0.24 : 1057.34
0.25 : 1093.31
0.26 : 1117.05
0.27 : 1136.58
0.28 : 1161.18
0.29 : 1182.22
0.3 : 1208.07
0.31 : 1236.69
0.32 : 1260.18
0.33 : 1280.94
0.34 : 1308.62
0.35 : 1324.48
0.36 : 1342.87
0.37 : 1369.54
0.38 : 1389.25
0.39 : 1417.95
0.4 : 1448.77
0.41 : 1474.67
0.42 : 1491.16
0.43 : 1511.26
0.44 : 1528.27
0.45 : 1551.32
0.46 : 1570.45
0.47 : 1593.69
0.48 : 1609.41
0.49 : 1632.22
0.5 : 1647.4
Simulation currently on network networkovermu030.gml : Name: 
Type: Graph
Number of nodes: 2500
Number of edges: 10283
Average degree:   8.2264...
Counter: 0, Centrality: modv_pos
0.01 : 143.28
0.02 : 349.42
0.03 : 394.47
0.04 : 480.14
0.05 : 535.33
0.06 : 601.61
0.07 : 652.09
0.08 : 720.15
0.09 : 781.18
0.1 : 814.12
0.11 : 854.43
0.12 : 893.92
0.13 : 928.58
0.14 : 958.86
0.15 : 992.71
0.16 : 1019.69
0.17 : 1060.9
0.18 : 1095.41

0.4 : 1490.9
0.41 : 1521.56
0.42 : 1538.38
0.43 : 1558.36
0.44 : 1579.28
0.45 : 1605.05
0.46 : 1620.36
0.47 : 1637.47
0.48 : 1653.71
0.49 : 1667.32
0.5 : 1679.1
Counter: 0, Centrality: map_eq
0.01 : 373.81
0.02 : 450.09
0.03 : 512.7
0.04 : 555.02
0.05 : 589.87
0.06 : 631.8
0.07 : 668.49
0.08 : 706.62
0.09 : 735.36
0.1 : 760.33
0.11 : 798.67
0.12 : 817.1
0.13 : 839.24
0.14 : 868.66
0.15 : 891.78
0.16 : 918.57
0.17 : 941.37
0.18 : 974.12
0.19 : 998.22
0.2 : 1020.75
0.21 : 1039.29
0.22 : 1071.61
0.23 : 1112.25
0.24 : 1131.96
0.25 : 1147.38
0.26 : 1163.84
0.27 : 1190.06
0.28 : 1219.03
0.29 : 1246.61
0.3 : 1277.2
0.31 : 1295.32
0.32 : 1308.91
0.33 : 1331.18
0.34 : 1346.58
0.35 : 1371.39
0.36 : 1385.44
0.37 : 1412.79
0.38 : 1441.48
0.39 : 1470.43
0.4 : 1485.34
0.41 : 1502.68
0.42 : 1515.94
0.43 : 1532.17
0.44 : 1547.79
0.45 : 1565.01
0.46 : 1581.21
0.47 : 1602.16
0.48 : 1619.76
0.49 : 1641.12
0.5 : 1666.92
Counter: 1, Centrality: modv_pos
0.01 : 130.54
0.02 : 235.42
0.03 : 348.77
0.04 : 424.

0.26 : 1132.49
0.27 : 1162.58
0.28 : 1196.48
0.29 : 1223.62
0.3 : 1239.52
0.31 : 1274.48
0.32 : 1301.38
0.33 : 1330.1
0.34 : 1358.1
0.35 : 1384.84
0.36 : 1408.26
0.37 : 1424.13
0.38 : 1449
0.39 : 1462.12
0.4 : 1486.26
0.41 : 1521.02
0.42 : 1539.66
0.43 : 1564.22
0.44 : 1584.57
0.45 : 1608.11
0.46 : 1623.59
0.47 : 1649.89
0.48 : 1669.15
0.49 : 1685.03
0.5 : 1694.43
Counter: 1, Centrality: map_eq
0.01 : 132.64
0.02 : 216.51
0.03 : 281.44
0.04 : 342.81
0.05 : 384.07
0.06 : 442.66
0.07 : 479.62
0.08 : 523.96
0.09 : 562.46
0.1 : 608.26
0.11 : 638.82
0.12 : 678.45
0.13 : 704.85
0.14 : 734.37
0.15 : 774.19
0.16 : 801.01
0.17 : 836.91
0.18 : 863.77
0.19 : 903.47
0.2 : 927.63
0.21 : 960.41
0.22 : 995.73
0.23 : 1040.92
0.24 : 1066.13
0.25 : 1088.9
0.26 : 1114.39
0.27 : 1141.02
0.28 : 1172.15
0.29 : 1201.26
0.3 : 1240.84
0.31 : 1260.87
0.32 : 1281.33
0.33 : 1302.27
0.34 : 1328.67
0.35 : 1353.2
0.36 : 1371.45
0.37 : 1399.57
0.38 : 1434.79
0.39 : 1460.82
0.4 : 1482.33
0.41 : 1496.7
0.42 : 1521.17
0

...

0.13 : 798.83
0.14 : 829.82
0.15 : 858.01
0.16 : 895.05
0.17 : 926.57
0.18 : 953.98
0.19 : 980.96
0.2 : 1012.2
0.21 : 1042.62
0.22 : 1070.11
0.23 : 1094.29
0.24 : 1121.6
0.25 : 1146.85
0.26 : 1174.27
0.27 : 1191.24
0.28 : 1222.43
0.29 : 1249.32
0.3 : 1266.4
0.31 : 1295.84
0.32 : 1311.44
0.33 : 1344.2
0.34 : 1368.25
0.35 : 1388.42
0.36 : 1415.91
0.37 : 1438.68
0.38 : 1458.52
0.39 : 1478.38
0.4 : 1499.6
0.41 : 1522.89
0.42 : 1548.18
0.43 : 1569.56
0.44 : 1587.76
0.45 : 1608.69
0.46 : 1627.92
0.47 : 1647.77
0.48 : 1668.46
0.49 : 1692.73
0.5 : 1708.59
Counter: 1, Centrality: modv_neg
0.01 : 157.28
0.02 : 245.64
0.03 : 336.4
0.04 : 405.2
0.05 : 473.78
0.06 : 532.85
0.07 : 579.9
0.08 : 642.3
0.09 : 677.71
0.1 : 724.83
0.11 : 776.66
0.12 : 814.78
0.13 : 841.69
0.14 : 885.51
0.15 : 917.99
0.16 : 966.21
0.17 : 1003.13
0.18 : 1043.9
0.19 : 1069.34
0.2 : 1101.08
0.21 : 1122.11
0.22 : 1150
0.23 : 1179.91
0.24 : 1222.58
0.25 : 1248.69
0.26 : 1280.1
0.27 : 1313.38
0.28 : 1352.22
0.29 : 1377.64
0.3 :

0.4 : 1413.63
0.41 : 1433.36
0.42 : 1447.42
0.43 : 1462.11
0.44 : 1481.2
0.45 : 1495.4
0.46 : 1523.67
0.47 : 1551.59
0.48 : 1567.01
0.49 : 1588.61
0.5 : 1609.55
Counter: 0, Centrality: cbc
0.01 : 307.6
0.02 : 387.24
0.03 : 429.52
0.04 : 462.76
0.05 : 501.07
0.06 : 524.35
0.07 : 548.26
0.08 : 577.34
0.09 : 607.66
0.1 : 633.51
0.11 : 655.85
0.12 : 681.82
0.13 : 713.13
0.14 : 730.85
0.15 : 764.52
0.16 : 785.7
0.17 : 807.53
0.18 : 830.03
0.19 : 848.63
0.2 : 871.27
0.21 : 889.26
0.22 : 912.31
0.23 : 930.91
0.24 : 961.47
0.25 : 986.78
0.26 : 1008.36
0.27 : 1037.87
0.28 : 1059.26
0.29 : 1088.77
0.3 : 1110.38
0.31 : 1134.67
0.32 : 1153.92
0.33 : 1187.04
0.34 : 1211.71
0.35 : 1233.29
0.36 : 1262.71
0.37 : 1287.13
0.38 : 1312.04
0.39 : 1331.83
0.4 : 1363.93
0.41 : 1385.77
0.42 : 1410.5
0.43 : 1438.37
0.44 : 1461.99
0.45 : 1483.88
0.46 : 1507.48
0.47 : 1533.66
0.48 : 1558.46
0.49 : 1577.24
0.5 : 1602.1
Counter: 0, Centrality: modv_neg
0.01 : 317.91
0.02 : 384.42
0.03 : 436.43
0.04 : 461.88
0.05 :

0.29 : 1149.95
0.3 : 1182.57
0.31 : 1207.87
0.32 : 1236.78
0.33 : 1267
0.34 : 1293.36
0.35 : 1327.15
0.36 : 1354.62
0.37 : 1384.72
0.38 : 1400.7
0.39 : 1427.88
0.4 : 1447.44
0.41 : 1465.14
0.42 : 1483.25
0.43 : 1498.35
0.44 : 1517.09
0.45 : 1544.22
0.46 : 1570.53
0.47 : 1594.08
0.48 : 1620.4
0.49 : 1642.34
0.5 : 1669.24
Counter: 1, Centrality: cbc
0.01 : 110.68
0.02 : 178.99
0.03 : 246.55
0.04 : 289.24
0.05 : 329.4
0.06 : 366.7
0.07 : 408.95
0.08 : 444.31
0.09 : 470.12
0.1 : 511.71
0.11 : 537.24
0.12 : 579.75
0.13 : 608.18
0.14 : 635.84
0.15 : 673.33
0.16 : 703.42
0.17 : 732.6
0.18 : 758.55
0.19 : 788.49
0.2 : 812.08
0.21 : 841.49
0.22 : 875.81
0.23 : 903.18
0.24 : 931.72
0.25 : 953.71
0.26 : 988.75
0.27 : 1025.39
0.28 : 1052.83
0.29 : 1074.64
0.3 : 1118.36
0.31 : 1139.29
0.32 : 1166.55
0.33 : 1197.68
0.34 : 1223.31
0.35 : 1257.79
0.36 : 1285.29
0.37 : 1316.24
0.38 : 1347.87
0.39 : 1380.84
0.4 : 1410.85
0.41 : 1431.68
0.42 : 1462.38
0.43 : 1496.18
0.44 : 1523.14
0.45 : 1554.23
0.46 : 1

0.05 : 509.92
0.06 : 536.12
0.07 : 561.22
0.08 : 614.41
0.09 : 644.47
0.1 : 669.48
0.11 : 691.65
0.12 : 720.59
0.13 : 747.27
0.14 : 761.83
0.15 : 782.28
0.16 : 806.44
0.17 : 822.5
0.18 : 830.1
0.19 : 855.48
0.2 : 888.18
0.21 : 915.66
0.22 : 948.85
0.23 : 983.45
0.24 : 1005.62
0.25 : 1034.66
0.26 : 1052.78
0.27 : 1081.35
0.28 : 1098.31
0.29 : 1116.45
0.3 : 1134.04
0.31 : 1152.45
0.32 : 1177.04
0.33 : 1186.71
0.34 : 1203.67
0.35 : 1219.2
0.36 : 1234.96
0.37 : 1247.89
0.38 : 1273.02
0.39 : 1303.15
0.4 : 1326.72
0.41 : 1346.2
0.42 : 1378.3
0.43 : 1404.71
0.44 : 1424.81
0.45 : 1455.28
0.46 : 1484.02
0.47 : 1511.12
0.48 : 1534.41
0.49 : 1559.29
0.5 : 1581.92
Counter: 0, Centrality: cbc
0.01 : 239.15
0.02 : 347.77
0.03 : 390.92
0.04 : 435.46
0.05 : 453.48
0.06 : 489.98
0.07 : 521.85
0.08 : 554.15
0.09 : 588.92
0.1 : 610.4
0.11 : 633.75
0.12 : 664.45
0.13 : 699.04
0.14 : 716.86
0.15 : 742.47
0.16 : 763.94
0.17 : 785.09
0.18 : 807.73
0.19 : 832.94
0.2 : 861.47
0.21 : 882.75
0.22 : 905.11
0.23 :

0.45 : 1707.14
0.46 : 1731.77
0.47 : 1759.62
0.48 : 1782.89
0.49 : 1803.67
0.5 : 1824.1
Counter: 1, Centrality: kshell
0.01 : 78.09
0.02 : 143.72
0.03 : 193.53
0.04 : 264.3
0.05 : 313.26
0.06 : 355.54
0.07 : 393.28
0.08 : 451.12
0.09 : 497.38
0.1 : 535.91
0.11 : 562.75
0.12 : 598.47
0.13 : 627.05
0.14 : 659.27
0.15 : 682.05
0.16 : 710.79
0.17 : 732.59
0.18 : 756.27
0.19 : 782.12
0.2 : 819.5
0.21 : 859.06
0.22 : 900.88
0.23 : 933.58
0.24 : 962.83
0.25 : 996.23
0.26 : 1015.64
0.27 : 1043.93
0.28 : 1075.78
0.29 : 1092.82
0.3 : 1116.65
0.31 : 1138.06
0.32 : 1159.98
0.33 : 1185.02
0.34 : 1200.45
0.35 : 1224.15
0.36 : 1237.48
0.37 : 1252.55
0.38 : 1283.14
0.39 : 1314
0.4 : 1339.81
0.41 : 1374.58
0.42 : 1407.77
0.43 : 1433.99
0.44 : 1462.82
0.45 : 1489.81
0.46 : 1514.09
0.47 : 1548.46
0.48 : 1569.57
0.49 : 1599.95
0.5 : 1628.46
Counter: 1, Centrality: cbc
0.01 : 90.71
0.02 : 159.85
0.03 : 213.96
0.04 : 257.09
0.05 : 303.21
0.06 : 336.33
0.07 : 381.01
0.08 : 408.99
0.09 : 444.5
0.1 : 484.78
0.